In [51]:
import pandas as pd
import numpy as np
from scipy import spatial
import math
import polars as pl

In [54]:
df = pd.read_pickle("input.pickle")
query_df = pd.read_pickle("query.pickle")


In [34]:
shots = df[["x","y"]].to_numpy()

In [35]:
shots

array([[69.63419583, 94.27175171],
       [47.8454922 , 66.00813963],
       [23.96564338, 52.0437865 ],
       ...,
       [55.34976791, 84.98903762],
       [ 1.64021229,  1.21028382],
       [90.58841013, 67.67529963]])

In [43]:
point = query_df.loc[0]

In [44]:
point

x    20.474901
y    77.058193
Name: 0, dtype: float64

In [45]:
distance = spatial.KDTree(shots).query(point)[0]

In [46]:
distance

0.07186779666948438

### This seems to work, have to create the algorithm for it

In [41]:
out = []
for idx, row in query_df.iterrows():

    diffs = (df[["x", "y"]] - row).pipe(lambda df: df**2).sum(axis=1)
    out.append({"dist": math.sqrt(diffs.min()), "weapon": df.loc[diffs.argmin(), "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out.csv", index=False)

In [42]:
%%timeit
out = []
for idx, row in query_df.iterrows():

    diffs = (df[["x", "y"]] - row).pipe(lambda df: df**2).sum(axis=1)
    out.append({"dist": math.sqrt(diffs.min()), "weapon": df.loc[diffs.argmin(), "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out.csv", index=False)

6.42 s ± 233 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
# This is 20x better than the original
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)


In [58]:
%%timeit
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)

329 ms ± 4.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


try with polar

In [52]:
df = pl.read_csv("input.csv")
query_df = pl.read_csv("query.csv")